In [21]:
import numpy as np


def f1(x):
    return abs(0.8 * (x[0]**2 + x[0] - 1) * x[2] + 0.12 * x[0]**2 + 2.16 * x[0] - 0.12)

def f2(x):
    return abs((1 + x[0]**2) * x[3] + 0.4 * x[0]**2 - 1.6 * x[0] - 0.4)

def f3(x):
    return abs((1 + x[0]**2) * x[4] + x[0]**2 - 1)

def f4(x):
    return abs((1 + x[0]**2) * x[5] + 0.8 * (x[0]**2 + x[0] - 1))

def f5(x):
    return abs(x[2] * x[6] - 0.02 * x[5] - x[4] - x[2] * x[3] - 0.16 * x[3])

def f6(x):
    return abs(x[6]**2 - 2 * x[3] * x[6] + x[5]**2 + x[3]**2 - x[1]**2)

def f7(x):
    return abs(x[7] - x[1] * x[2])

def f8(x):
    return abs(0.0476 * x[2] * x[7]**12 + x[2] - 2.104)


LB = np.array([-3, -1, -2, -1, -1, -0.5, -1.5, -1.5])
UB = np.array([1, 1, 2, 1, 1, 0.5, 1.5, 1.5])

In [22]:
def penalty(x):
    penalties = [f2(x), f3(x), f4(x), f5(x), f6(x), f7(x), f8(x)]
    return sum(p**2 for p in penalties)

def objective(x):
    return f1(x) + penalty(x)

In [23]:
def initialize_population(population_size, num_genes):
    return np.random.uniform(low=LB, high=UB, size=(population_size, num_genes))

def tournament_selection(population, tournament_size):
    idx = np.random.choice(len(population), tournament_size, replace=False)
    tournament = population[idx]
    return tournament[np.argmin([objective(individual) for individual in tournament])]

def arithmetic_crossover(parent1, parent2, crossover_rate):
    if np.random.rand() < crossover_rate:
        offspring1, offspring2 = parent1.copy(), parent2.copy()
        alpha = np.random.uniform(-0.5, 1.5, size=len(parent1))
        offspring1 = alpha * parent1 + (1 - alpha) * parent2
        offspring2 = alpha * parent2 + (1 - alpha) * parent1
        return offspring1, offspring2
    else:
        return parent1, parent2

def gaussian_mutation(offspring, mutation_rate, mutation_scale):
    for idx in range(len(offspring)):
        if np.random.rand() < mutation_rate:
            random_value = np.random.normal(loc=0, scale=mutation_scale, size=1)
            offspring[idx] += random_value
            offspring[idx] = np.clip(offspring[idx], LB[idx], UB[idx])
    return offspring

In [24]:
def run_genetic_algorithm(num_runs, population_size, num_genes, num_generations, tournament_size,
                          crossover_rate, mutation_rate, mutation_scale):
    all_metrics = []

    for run in range(num_runs):

        population = initialize_population(population_size, num_genes)
        best_fitness = np.inf
        best_solution = None
        fitnesses = []

        for generation in range(num_generations):

            fitness = np.array([objective(individual) for individual in population])
            fitnesses.append(fitness)


            min_fitness_idx = np.argmin(fitness)
            if fitness[min_fitness_idx] < best_fitness:
                best_fitness = fitness[min_fitness_idx]
                best_solution = population[min_fitness_idx]


            parents = [tournament_selection(population, tournament_size) for _ in range(population_size)]


            new_population = []
            for i in range(0, population_size, 2):
                parent1, parent2 = parents[i], parents[i + 1]
                offspring1, offspring2 = arithmetic_crossover(parent1, parent2, crossover_rate)
                offspring1 = gaussian_mutation(offspring1, mutation_rate, mutation_scale)
                offspring2 = gaussian_mutation(offspring2, mutation_rate, mutation_scale)
                new_population.extend([offspring1, offspring2])


            population = np.array(new_population)[:population_size]

        
        run_metrics = {
            'Run': run + 1,
            'Best Fitness': np.min(fitnesses),
            'Worst Fitness': np.max(fitnesses),
            'Mean Fitness': np.mean(fitnesses),
            'Std Dev Fitness': np.std(fitnesses),
            'Median Fitness': np.median(fitnesses)
        }
        all_metrics.append(run_metrics)


        print(f"Run {run + 1}: Best Fitness = {best_fitness}, Best Solution = {best_solution}")


    print("\nMetrics Table:")
    print("-" * 110)
    print(f"{'Run':<5} | {'Best Fitness':<15} | {'Worst Fitness':<15} | {'Mean Fitness':<15} | {'Std Dev Fitness':<20} | {'Median Fitness':<15}")
    print("-" * 110)
    for result in all_metrics:
        print(f"{result['Run']:<5} | {result['Best Fitness']:<15.6f} | {result['Worst Fitness']:<15.6f} | {result['Mean Fitness']:<15.6f} | {result['Std Dev Fitness']:<20.6f} | {result['Median Fitness']:<15.6f}")
    print("-" * 110)

    
    all_best_fitnesses = np.array([result['Best Fitness'] for result in all_metrics])
    all_worst_fitnesses = np.array([result['Worst Fitness'] for result in all_metrics])
    all_mean_fitnesses = np.array([result['Mean Fitness'] for result in all_metrics])
    all_std_fitnesses = np.array([result['Std Dev Fitness'] for result in all_metrics])
    all_median_fitnesses = np.array([result['Median Fitness'] for result in all_metrics])

    overall_best_fitness = np.min(all_best_fitnesses)
    overall_worst_fitness = np.max(all_worst_fitnesses)
    overall_mean_fitness = np.mean(all_mean_fitnesses)
    overall_std_fitness = np.sqrt(np.sum(np.square(all_std_fitnesses)) / len(all_std_fitnesses))
    overall_median_fitness = np.median(all_median_fitnesses)

    
    print("-" * 110)
    print(f"{'Overall':<5} | {overall_best_fitness:<15.6f} | {overall_worst_fitness:<15.6f} | {overall_mean_fitness:<15.6f} | {overall_std_fitness:<20.6f} | {overall_median_fitness:<15.6f}")
    print("-" * 110)

In [25]:
num_runs = 30
population_size = 200
num_genes = len(LB)
num_generations = 500
tournament_size = 5
crossover_rate = 0.7
mutation_rate = 0.05
mutation_scale = 0.05

In [26]:
run_genetic_algorithm(num_runs, population_size, num_genes, num_generations, tournament_size,
                      crossover_rate, mutation_rate, mutation_scale)

C:\Users\pedro\AppData\Local\Temp\ipykernel_4784\3370216165.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  offspring[idx] += random_value


Run 1: Best Fitness = 0.0008259636889077764, Best Solution = [ 0.38838265 -0.51229355  1.99936363  0.83405284  0.73130443  0.31073606
  1.26533304 -1.01395312]
Run 2: Best Fitness = 0.0019561053075348484, Best Solution = [0.39193946 0.48237302 2.04927126 0.83552935 0.72431839 0.30108852
 1.25107629 0.97408499]
Run 3: Best Fitness = 0.0019981540077281357, Best Solution = [ 0.39203056 -0.4815267   2.05056935  0.83533505  0.72404842  0.30085452
  1.25036343 -0.97289473]
Run 4: Best Fitness = 0.004066534095070988, Best Solution = [0.39510697 0.45272901 2.09500849 0.83630193 0.71780395 0.29213563
 1.2379182  0.93051921]
Run 5: Best Fitness = 0.0010098355972644976, Best Solution = [ 0.38917443 -0.50593859  2.01034415  0.83406458  0.72965952  0.30868972
  1.2616747  -1.00595375]
Run 6: Best Fitness = 0.0006901374569153522, Best Solution = [0.36454207 0.66514098 1.69966837 0.82221815 0.77318629 0.36394369
 1.36468708 1.14279665]
Run 7: Best Fitness = 0.0009719587213973082, Best Solution = [ 0.